## 네이버 블로그에서 검색어로 링크 수집 (여러 페이지) 

### 네이버 블로그에서 검색어로 링크 수집하는 함수 생성

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import json
import pandas as pd
import os, tqdm, time

# 네이버 블로그 검색 결과 데이터프레임을 반환하는 함수를 생성합니다.
def getBlogDf(searchWord, bgnDate, endDate, page = 1):
    
    # 요청 URL을 설정합니다.
    url = 'https://section.blog.naver.com/ajax/SearchList.naver'
    
    # 쿼리 스트링을 딕셔너리로 설정합니다.
    query = {
        'countPerPage': 7,
        'currentPage': page,
        'keyword': searchWord,
        'startDate': bgnDate, 
        'endDate': endDate, 
        'orderBy': 'sim',
        'type': 'post'
    }
    
    # 요청 헤더를 딕셔너리로 설정합니다
    headers = {
        'content-type': 'application/json;charset=UTF-8',
        'referer': 'https://section.blog.naver.com/Search/Post.naver',
        'user-agent': 'Mozilla/5.0 Chrome/99.0.4844.83 Safari/537.36'
    }
    
    # HTTP 요청을 실행합니다. 
    res = requests.get(url = url, params = query, headers = headers)
    
    # HTTP 응답 상태코드가 정상이면 데이터를 수집합니다.
    if res.status_code == 200:
                
        # HTTP 응답 바디를 딕셔너리로 변환합니다. 
        data = json.loads(s = res.text.replace(')]}\',', ''))
        
        # data에서 검색 결과 개수를 totalCnt에 할당합니다.
        totalCnt = data['result']['totalCount']
        
        # totalCnt가 1 이상이면 데이터프레임을 생성합니다.
        if totalCnt >= 1:
            
            # data['result']에서 검색 결과 데이터를 데이터프레임으로 변환합니다.
            df = pd.DataFrame(data = data['result']['searchList'])
            
            # 필요한 열이름만 선택합니다.
            cols = ['blogId', 'logNo', 'blogName', 'nickName', 'noTagTitle', 'addDate']
            df = df[cols]
            
            # logNo를 문자열로 변환합니다.
            df['logNo'] = df['logNo'].astype('str')
            
            # addDate를 날짜 데이터로 변환합니다.
            df['addDate'] = pd.to_datetime(arg = df['addDate'], unit = 'ms')
            
            # df를 반환합니다.
            return df
        
        # totalCnt가 0이면 NULL을 반환합니다.
        else:
            
            # None을 반환합니다.
            return None
    
    # HTTP 응답 상태코드가 정상이 아니면 에러 메시지를 반환합니다.
    else:
        
        print('HTTP 응답 상태코드는 200이 아닙니다!')
        

In [ ]:
# 2 페이지로 테스트합니다.
test = getBlogDf(
    searchWord = '스탠바이미', 
    bgnDate = '2022-04-01', 
    endDate = '2022-04-05', 
    page = 2
)

In [ ]:
# test를 확인합니다.
test

### for 반복문 실행

In [ ]:
# 최종 결과를 저장할 빈 데이터프레임을 생성합니다.
blog = pd.DataFrame()

In [ ]:
# for 반복문을 실행하여 5 페이지까지 수집합니다.
for i in tqdm.tqdm_notebook(range(5)):
    
    # 지정된 페이지의 데이터프레임을 수집합니다.
    df = getBlogDf(
        searchWord = '스탠바이미', 
        bgnDate = '2022-04-01', 
        endDate = '2022-04-05', 
        page = i + 1
    )
    
    # blog에 df를 추가합니다.
    blog = pd.concat(objs = [blog, df])
    
    # 1초간 멈춥니다.
    time.sleep(1)

In [ ]:
# blog의 처음 5행을 출력합니다.
blog.head()

In [ ]:
# blog의 정보를 확인합니다.
blog.info()

### xlsx 파일로 저장

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# xlsx 파일명을 설정합니다.
fileName = 'Naver Blog Links.xlsx'

In [ ]:
# xlsx 파일로 저장합니다.
blog.to_excel(excel_writer = fileName, index = None)

## End of Document